Librerias

In [2]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.1.0


Lectura de dataset

Para la lectura del modelo necesitamos armar un dataset que contenga: 
CDD1 
   - Diffx 
   - Diffy
   - EventNumber
   - Angulo


x : [Diffx, Diffy] (filtrado por Eventos) 

y : [cos(Angulo),sin(Angulo)]

Pruebas: 
   1. Con la información de Diffx e Diffy tratar de identificar el angulo. (más rápido) 
   2. Reconstruir la imagen con Diffx e Diffy, reemplazar las capas iniciales por capas convolucionales. (más lento) 

In [ ]:
# Realizar la lectura de los datos y armar la esstructura dejar en x e y



Normalizar los datos

Separar en x_train, y_train , x_test,y_test 

Modelo

In [5]:
input_shape_model = 200
def build_model():
  model = keras.Sequential([
    
    #
      
    # Capas densas "normales"
    layers.Dense(64, activation='relu', input_shape=[input_shape_model]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(2)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                12864     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 66        
Total params: 27,490
Trainable params: 27,490
Non-trainable params: 0
____________________________________________________

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

Graficos de entrenamiento

Salida